最佳实践是为向量数据库提供高质量的问答对。

# 文档加载

## 文件目录

In [10]:
pip show unstructured

Name: unstructured
Version: 0.12.2
Summary: A library that prepares raw documents for downstream ML tasks.
Home-page: https://github.com/Unstructured-IO/unstructured
Author: Unstructured Technologies
Author-email: devops@unstructuredai.io
License: Apache-2.0
Location: /Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/md-8WLN4Vov-py3.10/lib/python3.10/site-packages
Requires: backoff, beautifulsoup4, chardet, dataclasses-json, emoji, filetype, langdetect, lxml, nltk, numpy, python-iso639, python-magic, rapidfuzz, requests, tabulate, typing-extensions, unstructured-client, wrapt
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!poetry add "unstructured[all-docs]"

Using version ^0.12.2 for unstructured

Updating dependencies
Resolving dependencies... (30.2s)://files.pythonhosted.org/packages/56/fc/a3c13ded7b3057680c8ae95a9b6cc83e63657c38e0005c400a5d018a33a7/pyreadline3-3.4.1-py3-none-any.whl (23.5s)m9;2m(14.9s)Resolving dependencies... Downloading https://files.pythonhosted.org/packages/5e/55/7a85d92b115bb530797cd2ac5c11b905acbfd1d71cea88207eb276d1506a/pypdfium2-4.26.0-py3-none-macosx_10_13_x86_64.whl  22% (10.8s)Resolving dependencies... (16.4s)Resolving dependencies... (17.1s)Resolving dependencies... (19.1s)

Package operations: 57 installs, 0 updates, 0 removals

  • Installing mpmath (1.3.0): Pending...
  • Installing mpmath (1.3.0): Installing...
  • Installing mpmath (1.3.0)
  • Installing filelock (3.13.1): Pending...
  • Installing fsspec (2023.12.2): Pending...
  • Installing networkx (3.2.1): Pending...
  • Installing sympy (1.12): Pending...
  • Installing networkx (3.2.1): Pending...
  • Installing sympy (1.12): Pending...
  • Insta

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./', glob="**/*.md")
docs = loader.load()

## 加载CSV

## 加载HTML

In [5]:
!poetry add parser


Could not find a matching version of package parser


In [14]:
!poetry add nest_asyncio

Using version ^1.6.0 for nest-asyncio

Updating dependencies
Resolving dependencies... (10.4s)Resolving dependencies... (7.4s)Resolving dependencies... (7.6s)

No dependencies to install or update

Writing lock file


In [16]:
# 仅在jupyter中需要
import nest_asyncio
nest_asyncio.apply()

In [37]:
from bs4 import BeautifulSoup, SoupStrainer
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_core.utils.html import PREFIXES_TO_IGNORE_REGEX, SUFFIXES_TO_IGNORE_REGEX
from parser import langchain_docs_extractor
import re

In [38]:
# 提取langchain的Docs文档
def metadata_extractor(meta: dict, soup: BeautifulSoup) -> dict:
    title = soup.find("title")
    description = soup.find("meta", attrs={"name": "description"})
    html = soup.find("html")
    return {
        "source": meta["loc"],
        "title": title.get_text() if title else "",
        "description": description.get("content", "") if description else "",
        "language": html.get("lang", "") if html else "",
        **meta,
    }

def load_langchain_docs():
    return SitemapLoader(
        "https://python.langchain.com/sitemap.xml",
        filter_urls=["https://python.langchain.com/"],
        parsing_function=langchain_docs_extractor,
        default_parser="lxml",
        bs_kwargs={
            "parse_only": SoupStrainer(
                name=("article", "title", "html", "lang", "content")
            ),
        },
        meta_function=metadata_extractor,
    ).load()

In [39]:
# 提取langchain的API文档
def simple_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()

def load_api_docs():
    return RecursiveUrlLoader(
        url="https://api.python.langchain.com/en/latest/",
        max_depth=8,
        extractor=simple_extractor,
        prevent_outside=True,
        use_async=True,
        timeout=600,
        # Drop trailing / to avoid duplicate pages.
        link_regex=(
            f"href=[\"']{PREFIXES_TO_IGNORE_REGEX}((?:{SUFFIXES_TO_IGNORE_REGEX}.)*?)"
            r"(?:[\#'\"]|\/[\#'\"])"
        ),
        check_response_status=True,
        exclude_dirs=(
            "https://api.python.langchain.com/en/latest/_sources",
            "https://api.python.langchain.com/en/latest/_modules",
        ),
    ).load()

In [40]:
# langsmith的docs文档
def load_langsmith_docs():
    return RecursiveUrlLoader(
        url="https://docs.smith.langchain.com/",
        max_depth=8,
        extractor=simple_extractor,
        prevent_outside=True,
        use_async=True,
        timeout=600,
        # Drop trailing / to avoid duplicate pages.
        link_regex=(
            f"href=[\"']{PREFIXES_TO_IGNORE_REGEX}((?:{SUFFIXES_TO_IGNORE_REGEX}.)*?)"
            r"(?:[\#'\"]|\/[\#'\"])"
        ),
        check_response_status=True,
    ).load()

In [ ]:
langchain_docs = await load_langchain_docs()

In [ ]:
langchain_api = await load_api_docs()

In [ ]:
langchain_api

## 加载JSON

## 加载Markdown

In [152]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./index.md")
loader.load()

[Document(page_content='ok', metadata={'source': './index.md'})]

## 加载PDF

# 文本切分

## HTMLHeaderTextSplitter

In [2]:
from langchain.text_splitter import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

[Document(page_content='Foo'),
 Document(page_content='Some intro text about Foo.  \nBar main section Bar subsection 1 Bar subsection 2', metadata={'Header 1': 'Foo'}),
 Document(page_content='Some intro text about Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section'}),
 Document(page_content='Some text about the first subtopic of Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 1'}),
 Document(page_content='Some text about the second subtopic of Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 2'}),
 Document(page_content='Baz', metadata={'Header 1': 'Foo'}),
 Document(page_content='Some text about Baz', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}),
 Document(page_content='Some concluding text about Foo', metadata={'Header 1': 'Foo'})]

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

url = "http://www.hongmeng-info.com/"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)
splits[:10]

[Document(page_content='Toggle navigation  \n首页 互联网应用 信息化服务 电子税务 招聘 联系我们  \nPrevious Next  \n稳健、高效 人性化的电子竞价系统, 千亿级股权交易平台实践检验'),
 Document(page_content='鸿蒙在线竞价系统，微信，APP，智能终端，多媒体控制.  \n了解更多>>', metadata={'Header 2': '稳健、高效 人性化的电子竞价系统, 千亿级股权交易平台实践检验'}),
 Document(page_content='会员平台及CRM管理'),
 Document(page_content='社群运营的基础架构系统，支持复杂权益，积分管理，“会员卡”系统，权益/积分商城应用，社群用户关系管理，多种智能行为数据模型，面向客户群/社群运营者提供有效的解决方案.  \n详情 »', metadata={'Header 2': '会员平台及CRM管理'}),
 Document(page_content='互联网运营平台'),
 Document(page_content='核心组件系统支撑O2O类运营体系，订单系统，合作商/供应商/渠道商管理及结算系统，客服系统，营销支撑与分析，活动及传播系统，为运营提供有效灵活的支撑.  \n详情 »', metadata={'Header 2': '互联网运营平台'}),
 Document(page_content='电子商城'),
 Document(page_content='服务/产品的线上交易平台。根据你的需要，实现你想要的电子商城系统。模块化组合实现满足不同运营方多层次的系统需要.  \n详情 »', metadata={'Header 2': '电子商城'}),
 Document(page_content='数据技术&服务'),
 Document(page_content='清洗，分析，挖掘，分析。经验和自有的工具务实有效的解决深层次运营问题。我们擅长解决各种类型的数据接口.  \n详情 »', metadata={'Header 2': '数据技术&服务'})]

# 向量编码

In [5]:
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()

In [6]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [7]:
embedded_query = embeddings_model.embed_query("对话中提及的名字是什么?")
embedded_query[:5]

[-0.0037282993122298882,
 -0.01327033122820681,
 0.03234768953778434,
 0.0035204264887496776,
 -0.017729538998712827]

**CacheBackedEmbeddings**：支持缓存

# 向量存储

## 资料准备

In [17]:
!poetry add importlib

Using version ^1.0.4 for importlib

Updating dependencies
Resolving dependencies... (28.8s)://files.pythonhosted.org/packages/31/77/3781f65cafe55480b56914def99022a5d2965a4bb269655c89ef2f1de3cd/importlib-1.0.4.zip (0.6s)Resolving dependencies... (6.9s)Resolving dependencies... (10.3s)Resolving dependencies... (20.0s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing importlib (1.0.4): Pending...
  • Installing importlib (1.0.4): Downloading... 0%
  • Installing importlib (1.0.4): Downloading... 100%
  • Installing importlib (1.0.4): Preparing...
  • Installing importlib (1.0.4): Installing...
  • Installing importlib (1.0.4)

Writing lock file


In [30]:
import os
import importlib.util
spec = importlib.util.find_spec('langchain')
langchain_files_path = os.path.join(os.path.dirname(spec.origin), "docs/docs/modules")
print(langchain_files_path)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/md-8WLN4Vov-py3.10/lib/python3.10/site-packages/langchain/docs/docs/modules


## Chroma

In [8]:
!poetry add chromadb

Using version ^0.4.22 for chromadb

Updating dependencies
Resolving dependencies... (63.2s)://files.pythonhosted.org/packages/79/4d/9cc401e7b07e80532ebc8c8e993f42541534da9e9249c59ee0139dcb0352/websockets-12.0-py3-none-any.whl (55.6s)39m (36.2s)

Package operations: 41 installs, 1 update, 0 removals

  • Installing zipp (3.17.0): Pending...
  • Installing zipp (3.17.0): Installing...
  • Installing zipp (3.17.0)
  • Installing importlib-metadata (6.11.0): Pending...
  • Installing importlib-metadata (6.11.0): Downloading... 0%
  • Installing importlib-metadata (6.11.0): Downloading... 100%
  • Installing importlib-metadata (6.11.0): Installing...
  • Installing importlib-metadata (6.11.0)
  • Installing opentelemetry-api (1.22.0): Pending...
  • Installing pyasn1 (0.5.1): Pending...
  • Updating setuptools (69.0.2 -> 69.0.3): Pending...
  • Installing pyasn1 (0.5.1): Pending...
  • Updating setuptools (69.0.2 -> 69.0.3): Pending...
  • Installing opentelemetry-api (1.22.0): Downloading.

In [37]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
file = 'state_of_the_union.txt'
print(file)
raw_documents = TextLoader(file).load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

state_of_the_union.txt


用**字符串参数**做相似性查询

In [39]:
query = "总统关于 Ketanji Brown Jackson 的发言"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


用**向量参数**做相似性查询（可以做更深度优化，例如减少向量编码的事件）

In [40]:
embedding_vector = OpenAIEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## FAISS

In [72]:
!poetry add faiss-cpu

Using version ^1.7.4 for faiss-cpu

Updating dependencies
Resolving dependencies... (12.4s)://files.pythonhosted.org/packages/e4/5d/c35f5285b85b54b4b154ce40a8810d57a306f2da4a9a58cb7498f9aefadb/faiss_cpu-1.7.4-cp310-cp310-macosx_10_9_x86_64.whl  99% (0.5s)Resolving dependencies... (3.3s)Resolving dependencies... (4.5s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing faiss-cpu (1.7.4): Pending...
  • Installing faiss-cpu (1.7.4): Downloading... 0%
  • Installing faiss-cpu (1.7.4): Downloading... 30%
  • Installing faiss-cpu (1.7.4): Downloading... 48%
  • Installing faiss-cpu (1.7.4): Downloading... 60%
  • Installing faiss-cpu (1.7.4): Downloading... 90%
  • Installing faiss-cpu (1.7.4): Downloading... 100%
  • Installing faiss-cpu (1.7.4): Installing...
  • Installing faiss-cpu (1.7.4)

Writing lock file


## lanceDB

In [42]:
!poetry add lancedb

Using version ^0.5.1 for lancedb

Updating dependencies
Resolving dependencies... (16.8s)://files.pythonhosted.org/packages/f6/f0/10642828a8dfb741e5f3fbaac830550a518a775c7fff6f04a007259b0548/py-1.11.0-py2.py3-none-any.whl (3.1s)Resolving dependencies... (8.0s)Resolving dependencies... (8.4s)Resolving dependencies... (9.8s)Resolving dependencies... (11.0s)Resolving dependencies... (12.0s)Resolving dependencies... (12.6s)Resolving dependencies... (13.7s)Resolving dependencies... (16.5s)

Package operations: 8 installs, 0 updates, 0 removals

  • Installing py (1.11.0): Pending...
  • Installing pyarrow (15.0.0): Pending...
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing py (1.11.0): Downloading... 0%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing py (1.11.0): Downloading... 100%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing pyarrow

In [44]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import LanceDB

embeddings = OpenAIEmbeddings()

import lancedb
db = lancedb.connect("/tmp/lancedb")

table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = LanceDB.from_documents(documents, OpenAIEmbeddings(), connection=table)

[2024-01-24T08:00:55Z WARN  lance::dataset] No existing dataset at /tmp/lancedb/my_table.lance, it will be created


使用**字符串**做相似性检索

In [48]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


使用**向量**做相似性检索

In [50]:
retriever = db.as_retriever()

# 向量检索：retriever

## RAG

In [52]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("总统发言提到了什么技术方面的内容?")


'总统发言提到了新兴技术和美国制造业方面的内容。'

## 定制Recevier

In [53]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List

class CustomRetriever(BaseRetriever):    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        return [Document(page_content=query)]

retriever = CustomRetriever()

retriever.get_relevant_documents("bar")

[Document(page_content='bar')]